In [1]:
pip install -U duckduckgo_search

Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: duckduckgo_search
    Found existing installation: duckduckgo_search 7.5.1
    Uninstalling duckduckgo_search-7.5.1:
      Successfully uninstalled duckduckgo_search-7.5.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade --quiet duckduckgo-search

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install urlopen

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
from duckduckgo_search import DDGS
from datetime import date
import time

# Fetch and parse the webpage
url = "https://collider.com/andrei-tarkovsky-movies-ranked-imdb/"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

# Extract page title
page_title = soup.title.get_text(strip=True)

# Find and download the large header image
image_url = soup.find('link', media="(min-width: 1024px)")['href']
image_path = "assets/images/tarkovsky-banner.jpg"

os.makedirs("assets/images", exist_ok=True)
response = requests.get(image_url)
with open(image_path, "wb") as img_file:
    img_file.write(response.content)

# Create directories
os.makedirs("_posts", exist_ok=True)
# Create _config.yml for Jekyll configuration
with open("_config.yml", "w", encoding="utf-8") as config_file:
    config_file.write(f"""\
title: "Top Tarkovsky Movies"
description: "A collection of Andrei Tarkovsky's movies"
theme: jekyll-theme-minimal
logo: "/assets/images/logo.png"
author:
  name: "Pavel Charheika"
  email: "pc451309@students.mimuw.edu.pl"
paginate: 5
permalink: /:title/
plugins:
  - jekyll-feed
  - jekyll-seo-tag
  - jekyll-sitemap
""")

# Prepare the movie data
data = []
movies = soup.find_all('h2')

for movie in movies:
    span = movie.find('span')
    if not span:
        continue  # Skip if there's no <span>

    rank = span.get_text(strip=True)    
    title_span = span.find_next_sibling()
    title = title_span.get_text(strip=True) if title_span else "Unknown"

    rating_tag = movie.find_next_sibling('h3')
    rating = rating_tag.get_text(strip=True) if rating_tag else "N/A"

    description_tag = movie.find_next_sibling('p')
    sentence1 = description_tag.get_text(strip=True) if description_tag else "No description"
    
    sentence2 = "No description"
    if description_tag:
        next_p = description_tag.find_next_sibling('p')
        if next_p:
            sentence2 = next_p.get_text(strip=True)
    
    
    time.sleep(2)
    # Search for IMDb link
    search_result = DDGS().text(f"{title} - IMDb", max_results=1)
    link = search_result[0]['href'] if search_result else "#"

    movie_info = {
        "Rank": rank,
        "Title": title,
        "Rating": rating,
        "Sentence1": sentence1,
        "Sentence2": sentence2,
        "Link": link
    }
    data.append(movie_info)
# Create index.md (Home page)
with open("index.md", "w", encoding="utf-8") as index_file:
    index_file.write(f"""\
---
layout: default
title: "{page_title}"
header:
  overlay_image: "/{image_path}"
  caption: "Andrei Tarkovsky's Best Movies"
---

## List of Tarkovsky's Movies

""")

    for movie in data:
        filename = f"_posts/{date.today()}-{movie['Title'].replace(' ', '_')}.md"

        # Create individual movie pages
        with open(filename, "w", encoding="utf-8") as post_file:
            post_file.write(f"""\
---
title: "{movie['Title']}"
date: {date.today()}
layout: post
header:
  overlay_image: "/{image_path}"
  caption: "A still from {movie['Title']}"---

## {movie['Rank']} - {movie['Title']}

**Rating:** {movie['Rating']}

{movie['Sentence1']}

{movie['Sentence2']}

[IMDb - {movie['Title']}]({movie['Link']})
""")

        # Add movie links to the index page
        index_file.write(f"- [{movie['Title']}]({filename.replace('_posts/', '')})\n")

print("Jekyll site files have been generated.")

Jekyll site files have been generated.
